In [1]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import pandas as pd
from pandas.core.internals.array_manager import NullArrayProxy
import time
import datetime

get qualifying info

In [154]:

qualifying = {'season': [],
        'round': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': [],
        'city':[],
        'q_date': []}

for year in list(range(2014,2023)):
    
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            qualifying['season'].append(int(item['season']))
        except:
            qualifying['season'].append(None)

        try:
            qualifying['round'].append(int(item['round']))
        except:
            qualifying['round'].append(None)

        try:
            qualifying['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            qualifying['circuit_id'].append(None)

        try:
            qualifying['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            qualifying['lat'].append(None)

        try:
            qualifying['long'].append(float(item['Circuit']['Location']['long']))
        except:
            qualifying['long'].append(None)

        try:
            qualifying['country'].append(item['Circuit']['Location']['country'])
        except:
            qualifying['country'].append(None)

        try:
            qualifying['city'].append(item['Circuit']['Location']['locality'])
        except:
            qualifying['city'].append(None)

        try:
            qualifying['q_date'].append(item['Qualifying']['date'])
        except:
            qualifying['q_date'].append((datetime.datetime.strptime(item['date'], "%Y-%m-%d")-datetime.timedelta(days=1)).strftime("%Y-%m-%d"))
        
qualifying = pd.DataFrame(qualifying)
print(qualifying.shape)

(182, 8)


get qualifying results

In [157]:
for index, row in qualifying.iterrows():
    url = 'https://ergast.com/api/f1/{}/{}/qualifying/1.json'.format(row['season'],row['round'])
    r = requests.get(url)
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
      for race in item['QualifyingResults']:
        if race['Driver']['givenName']:
          qualifying.loc[index, 'pole_position'] = race['Driver']['givenName']+' '+race['Driver']['familyName']
        else:
          qualifying.loc[index, 'pole_position'] = None

get driver table

In [158]:
drivers = {'driver_id': [],
            'driver' : []}

url = 'https://ergast.com/api/f1/2022/drivers.json'
r = requests.get(url)
json = r.json()
num = 0

for item in json['MRData']['DriverTable']['Drivers']:
    try:
      drivers['driver'].append(item['givenName']+' '+item['familyName'])
      drivers['driver_id'].append(num)
      num += 1
    except:
      drivers['driver'].append(None)
      drivers['driver_id'].append(None)

drivers = pd.DataFrame(drivers)

In [159]:
drivers.to_csv('drivers.csv') 

In [160]:
qualifying = pd.merge(qualifying, drivers, right_on=['driver'], left_on=['pole_position'], how='left')

get weather for qualifying date

In [164]:
for index, row in qualifying.iterrows():
  url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{}%2C{}/{}?unitGroup=metric&elements=temp%2Cwindspeed%2Cconditions&include=days&key=UHADRGQA7EE52B4HAATFN5A2Q&contentType=json'.format(row['lat'],row['long'],row['q_date'])
  r = requests.get(url)
  json = r.json()
  for item in json['days']:
        try:
            qualifying.loc[index, 'temp'] = item['temp']
        except:
            qualifying.loc[index, 'temp'] = None

        try:
            qualifying.loc[index, 'windspeed'] = item['windspeed']
        except:
            qualifying.loc[index, 'windspeed'] = None

        try:
            qualifying.loc[index, 'conditions'] = item['conditions']
        except:
            qualifying.loc[index, 'conditions'] = None


In [165]:
qualifying['conditions'] = qualifying['conditions'].str.split(', ')

In [166]:
for index, row  in qualifying.iterrows():
  if row['conditions'][0] != 'Rain':
    qualifying.loc[index, 'rain'] = 0
    qualifying.loc[index, 'weather'] = row['conditions'][0]
  else:
    qualifying.loc[index, 'rain'] = 1
    try:
      qualifying.loc[index, 'weather'] = row['conditions'][1]
    except:
      qualifying.loc[index, 'weather'] = 'Precipitation'

transform data

In [167]:
from sklearn.preprocessing import OneHotEncoder

ohe_weather = OneHotEncoder()
transformed_weather = ohe_weather.fit_transform(qualifying[['weather']])

In [168]:
ohe_weather.categories_[0]

array(['Clear', 'Overcast', 'Partially cloudy', 'Precipitation'],
      dtype=object)

In [169]:
qualifying[['clear', 'overcast', 'partially_cloudy', 'precipitation']] = transformed_weather.toarray()

In [170]:
lambda x: x*10 if x<2 else (x**2 if x<4 else x+10)

<function __main__.<lambda>>

In [171]:
qualifying['wind'] = [1 if x<12 else (2 if x>=12 and x<20 else (3 if x>=20 and x<30 else(4 if x>=30 and x<40 else 5))) for x in qualifying['windspeed']]

In [172]:
ohe_circutid = OneHotEncoder()
transformed_circutid = ohe_circutid.fit_transform(qualifying[['circuit_id']])

In [173]:
ohe_circutid.categories_[0]

array(['albert_park', 'americas', 'bahrain', 'baku', 'catalunya',
       'hockenheimring', 'hungaroring', 'imola', 'interlagos', 'istanbul',
       'jeddah', 'losail', 'marina_bay', 'miami', 'monaco', 'monza',
       'mugello', 'nurburgring', 'portimao', 'red_bull_ring', 'ricard',
       'rodriguez', 'sepang', 'shanghai', 'silverstone', 'sochi', 'spa',
       'suzuka', 'villeneuve', 'yas_marina', 'zandvoort'], dtype=object)

In [175]:
qualifying[['albert_park', 'americas', 'bahrain', 'baku', 'catalunya',
       'hockenheimring', 'hungaroring', 'imola', 'interlagos', 'istanbul',
       'jeddah', 'losail', 'marina_bay', 'miami', 'monaco', 'monza',
       'mugello', 'nurburgring', 'portimao', 'red_bull_ring', 'ricard',
       'rodriguez', 'sepang', 'shanghai', 'silverstone', 'sochi', 'spa',
       'suzuka', 'villeneuve', 'yas_marina', 'zandvoort']] = transformed_circutid.toarray()

In [ ]:
qualifying = qualifying.drop(['circuit_id', 'lat', 'long', 'country', 'city', 'q_date', 'windspeed', 'conditions', 'weather', 'driver', 'pole_position'], axis=1)

In [177]:
qualifying.columns

Index(['season', 'round', 'pole_position', 'driver_id', 'temp', 'rain',
       'clear', 'overcast', 'partially_cloudy', 'precipitation', 'wind',
       'albert_park', 'americas', 'bahrain', 'baku', 'catalunya',
       'hockenheimring', 'hungaroring', 'imola', 'interlagos', 'istanbul',
       'jeddah', 'losail', 'marina_bay', 'miami', 'monaco', 'monza', 'mugello',
       'nurburgring', 'portimao', 'red_bull_ring', 'ricard', 'rodriguez',
       'sepang', 'shanghai', 'silverstone', 'sochi', 'spa', 'suzuka',
       'villeneuve', 'yas_marina', 'zandvoort'],
      dtype='object')

In [179]:
qualifying = qualifying.sort_values(['season', 'round'])

In [183]:
qualifying = qualifying[:173]

In [194]:
qualifying = qualifying.dropna()

In [196]:
qualifying.to_csv('qualifying.csv') 